In [75]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
from data_loader.data_reader import *
from data_loader.datasets import ImageDataset
import model.model as model_arch
import model.model as model_loss
import model.model as model_metric
from parse_config import ConfigParser
from utils import prepare_device
import torch
import numpy as np
from trainer import Trainer
import argparse
import collections
from torchvision import datasets

In [81]:
img_dir = 'data/images/'
mask_dir = 'data/labels/'

dataset = ImageDataset(img_dir, mask_dir, transform  = None)

In [86]:
def train_one_epoch(model, criterion, optimizer, data_loader, lr_scheduler, device, epoch, use_swa):
    model.train()

    for image, target in data_loader:
        image, target = image.to(device), target.to(device)

        output = model(image)

        loss, iou = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lr_scheduler.step()
        
    if use_swa:
        optimizer.swap_swa_sgd()

def evaluate(model, criterion, data_loader, device, epoch = None):
    model.eval()


    with torch.no_grad():
        for image, target in data_loader:
            image, target = image.to(device), target.to(device)

            output = model(image)

            loss, iou = criterion(output, target)


def train(dataset):
    torch.cuda.empty_cache()


    train_sampler = torch.utils.data.RandomSampler(dataset)

    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=32,
        sampler=train_sampler,
        num_workers=0,
        # collate_fn=utils.collate_fn,
        drop_last=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model_arch.UNet(3,2)
    model.to(device)

    params_to_optimize = [
        {"params": [p for p in model.parameters()
                    if p.requires_grad]},
    ]

    optimizer = torch.optim.SGD(
        params_to_optimize,
        lr=0.001, momentum=0.1, weight_decay=0.1)

    criterion = torch.nn.CrossEntropyLoss()

    epochs = 10
    lr_scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lambda x: (1 - x / (len(data_loader) * epochs)) ** 0.9)


    for epoch in range(epochs):
        train_one_epoch(model, criterion, optimizer, data_loader,
                        lr_scheduler, device, epoch, False)
        evaluate(model, criterion, data_loader,device, epoch)



In [87]:
train(dataset)

data/images/Cars277.png
data/labels/Cars263.xml
data/images/Cars62.png
data/labels/Cars342.xml
data/images/Cars279.png
data/labels/Cars251.xml
data/images/Cars69.png
data/labels/Cars349.xml
data/images/Cars113.png
data/labels/Cars19.xml
data/images/Cars418.png
data/labels/Cars154.xml
data/images/Cars129.png
data/labels/Cars115.xml
data/images/Cars104.png
data/labels/Cars26.xml
data/images/Cars146.png
data/labels/Cars378.xml
data/images/Cars380.png
data/labels/Cars182.xml
data/images/Cars2.png
data/labels/Cars31.xml
data/images/Cars97.png
data/labels/Cars40.xml
data/images/Cars290.png
data/labels/Cars284.xml
data/images/Cars131.png
data/labels/Cars125.xml
data/images/Cars16.png
data/labels/Cars336.xml
data/images/Cars387.png
data/labels/Cars185.xml
data/images/Cars205.png
data/labels/Cars239.xml
data/images/Cars336.png
data/labels/Cars120.xml
data/images/Cars329.png
data/labels/Cars117.xml
data/images/Cars421.png
data/labels/Cars179.xml
data/images/Cars22.png
data/labels/Cars316.xml
dat

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[32, 256, 256, 3] to have 3 channels, but got 256 channels instead